In [35]:
from langchain.schema import HumanMessage, SystemMessage
import boto3
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from opensearchpy import OpenSearch, RequestsHttpConnection
import os
import boto3
import json
import sys
from langchain.schema import BaseRetriever, Document
from typing import List, Optional, Dict, Tuple
from langchain.prompts.prompt import PromptTemplate
import uuid
from datetime import datetime
from botocore.exceptions import ClientError
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from boto3.dynamodb.conditions import Key
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.schema import BaseMessage, HumanMessage, AIMessage
from langchain.schema import BaseChatMessageHistory
from langchain_aws import ChatBedrock

from langchain_core.runnables.history import RunnableWithMessageHistory
import time
from botocore.exceptions import ClientError

In [24]:
def insert_chat_message(session_id, message_content, role, timestamp=None):
    """
    Insert a chat message into the DynamoDB table.
    
    Args:
        session_id (str): The session identifier
        message_content (str): The content of the message
        role (str): The role of the message sender (e.g., 'human', 'ai')
        timestamp (str, optional): Message timestamp. Defaults to current time if None.
    
    Returns:
        dict: The response from DynamoDB
    """
    # Initialize DynamoDB client
    dynamodb = boto3.resource('dynamodb', region_name='us-west-2')
    table = dynamodb.Table('conversations-table')
    
    # Use current time if timestamp not provided
    if timestamp is None:
        sequence = int(time.time() * 1000)  # millisecond timestamp
        timestamp = datetime.utcnow().isoformat()
    else:
        # Convert timestamp to sequence number
        dt = datetime.fromisoformat(timestamp)
        sequence = int(dt.timestamp() * 1000)
    
    # Prepare item to insert
    item = {
        'session_id': session_id,
        'sequence': sequence,
        'role': role,
        'message': message_content,
        'timestamp': timestamp
    }
    
    try:
        # Insert the item into the table
        response = table.put_item(Item=item)
        print(f"Message inserted successfully: {sequence}")
        return response
    except ClientError as e:
        print(f"Error inserting message: {e.response['Error']['Message']}")
        raise

In [25]:
def get_chat_messages(session_id, ascending=True, limit=None):
    """
    Retrieve chat messages for a session, sorted by sequence.
    
    Args:
        session_id (str): The session identifier
        ascending (bool): Sort order - True for ascending (oldest first), False for descending (newest first)
        limit (int, optional): Maximum number of messages to retrieve
    
    Returns:
        list: List of message items
    """
    # Initialize DynamoDB client
    dynamodb = boto3.resource('dynamodb', region_name='us-west-2')
    table = dynamodb.Table('conversations-table')
    
    messages = []
    
    try:
        # Prepare query parameters
        query_params = {
            'KeyConditionExpression': Key('session_id').eq(session_id),
            'ScanIndexForward': ascending  # True for ascending, False for descending
        }
        
        # Add limit if specified
        if limit:
            query_params['Limit'] = limit
        
        # Query messages for the session, sorted by sequence
        response = table.query(**query_params)
        
        # Add results to our messages list
        messages.extend(response['Items'])
        
        # Handle pagination if there are more results and no limit specified
        while 'LastEvaluatedKey' in response and not limit:
            response = table.query(
                KeyConditionExpression=Key('session_id').eq(session_id),
                ScanIndexForward=ascending,
                ExclusiveStartKey=response['LastEvaluatedKey']
            )
            messages.extend(response['Items'])
        
        print(f"Found {len(messages)} messages for session {session_id}")
        return messages
        
    except ClientError as e:
        print(f"Error retrieving messages: {e.response['Error']['Message']}")
        return []

In [30]:
def delete_all_session_messages(session_id):
    """
    Delete all messages for a specific session.
    
    Args:
        session_id (str): The session identifier
    
    Returns:
        dict: Summary of deletion operation
    """
    # Initialize DynamoDB client
    dynamodb = boto3.resource('dynamodb', region_name='us-west-2')
    table = dynamodb.Table('conversations-table')
    
    try:
        # First, query to get all messages for the session
        response = table.query(
            KeyConditionExpression=Key('session_id').eq(session_id)
        )
        
        items = response['Items']
        
        # Handle pagination if there are more results
        while 'LastEvaluatedKey' in response:
            response = table.query(
                KeyConditionExpression=Key('session_id').eq(session_id),
                ExclusiveStartKey=response['LastEvaluatedKey']
            )
            items.extend(response['Items'])
        
        # If no items found, return early
        if not items:
            print(f"No messages found for session {session_id}")
            return {"deleted_count": 0}
        
        # Delete each item
        deleted_count = 0
        for item in items:
            table.delete_item(
                Key={
                    'session_id': session_id,
                    'sequence': item['sequence']
                }
            )
            deleted_count += 1
        
        print(f"Deleted {deleted_count} messages for session {session_id}")
        return {"deleted_count": deleted_count}
        
    except ClientError as e:
        print(f"Error deleting messages: {e.response['Error']['Message']}")
        raise

In [55]:
class DynamoDBHistory(BaseChatMessageHistory):
    def __init__(self, session_id: str, table_name: str = "conversations-table"):
        self.session_id = session_id
        self.table_name = table_name
        self.dynamodb = boto3.resource('dynamodb')
        self.table = self.dynamodb.Table(table_name)
    
    @property
    def messages(self) -> list[BaseMessage]:
        """DynamoDB에서 메시지 로드"""
        try:
            response = self.table.query(
                KeyConditionExpression=Key('session_id').eq(self.session_id),
                ScanIndexForward=True,  
                Limit=10,
            )
            
            messages = []
            for item in response['Items']:
                if item['role'] == 'human':
                    messages.append(HumanMessage(content=item['message']))
                elif item['role'] == 'ai':
                    messages.append(AIMessage(content=item['message']))
            
            return messages
        except Exception as e:
            print(f"메시지 로드 실패: {e}")
            return []
    
    def add_message(self, message: BaseMessage) -> None:
        """새 메시지를 DynamoDB에 저장"""
        sequence = int(time.time() * 1000)  # millisecond timestamp
        
        if isinstance(message, HumanMessage):
            role = 'human'
        elif isinstance(message, AIMessage):
            role = 'ai'
        else:
            return
        
        self.table.put_item(
            Item={
                'session_id': self.session_id,
                'sequence': sequence,
                'role': role,
                'message': message.content,
                'timestamp': datetime.now().isoformat()
            }
        )
    
    def clear(self) -> None:
        """대화 기록 삭제"""
        try:
            # Query all items for this session
            response = self.table.query(
                KeyConditionExpression=Key('session_id').eq(self.session_id)
            )
            
            # Delete each item
            for item in response['Items']:
                self.table.delete_item(
                    Key={
                        'session_id': item['session_id'],
                        'sequence': item['sequence']
                    }
                )
        except Exception as e:
            print(f"메시지 삭제 실패: {e}")

In [56]:
response1 = insert_chat_message(
    session_id="test-session-001",
    message_content="안녕하세요, 도움이 필요합니다.",
    role="human"
)

Message inserted successfully: 1762142131364


In [53]:
result=get_chat_messages(session_id="test-session-001")

Found 12 messages for session test-session-001


In [54]:
result

[{'message': '안녕하세요',
  'role': 'human',
  'session_id': 'test-session-001',
  'timestamp': '2025-11-03T03:50:23.729690',
  'sequence': Decimal('1762141823729')},
 {'message': '안녕하세요! 반갑습니다! 😊\n\n한국어로 인사해 주셔서 기쁘네요. 저는 Claude라고 하는 AI 어시스턴트입니다. 오늘 어떻게 지내고 계신가요? \n\n한국어로 대화하는 것을 좋아해서, 궁금한 것이 있으시거나 도움이 필요한 일이 있으시면 언제든지 말씀해 주세요. 한국 문화, 언어, 음식, K-pop, 드라마 등 다양한 주제에 대해서도 이야기할 수 있어요. \n\n무엇에 대해 이야기하고 싶으신가요?',
  'role': 'ai',
  'session_id': 'test-session-001',
  'timestamp': '2025-11-03T03:50:23.799757',
  'sequence': Decimal('1762141823799')},
 {'message': '안녕하세요',
  'role': 'human',
  'session_id': 'test-session-001',
  'timestamp': '2025-11-03T03:51:28.095440',
  'sequence': Decimal('1762141888095')},
 {'message': '안녕하세요! 반갑습니다! 😊\n\n한국어로 인사해 주셔서 기쁘네요. 저는 Claude라고 하는 AI 어시스턴트입니다. 오늘 어떻게 지내고 계신가요? \n\n한국어로 대화하는 것을 좋아해서, 궁금한 것이 있으시거나 도움이 필요한 일이 있으시면 언제든지 말씀해 주세요. 한국 문화, 언어, 음식, K-pop, 드라마 등 다양한 주제에 대해서도 이야기할 수 있어요. \n\n무엇에 대해 이야기하고 싶으신가요?',
  'role': 'ai',
  'session_id': 'test-session-001',


In [57]:
delete_all_session_messages("test-session-001")

Deleted 13 messages for session test-session-001


{'deleted_count': 13}

In [58]:
class AdvancedLLM:
    """고급 LLM 스트리밍 관리자"""
    
    def __init__(self, region_name: str = "us-west-2"):
        self.region_name = region_name
        self.model_id = None
        self.bedrock_client = self._setup_bedrock_client()
        self.llm = self._setup_llm()
    
    def _setup_bedrock_client(self):
        """Bedrock 클라이언트 설정"""
        try:
            return boto3.client(
                service_name='bedrock-runtime',
                region_name=self.region_name
            )
        except Exception as e:
            print(f"❌ Bedrock 클라이언트 초기화 실패: {e}")
            return None
    
    def _setup_llm(self, model_id="global.anthropic.claude-sonnet-4-20250514-v1:0"):
        """LLM 설정"""
        try:
            if not self.bedrock_client:
                print("❌ Bedrock 클라이언트가 없습니다.")
                return None
            self.model_id = model_id
            return ChatBedrock(
                client=self.bedrock_client,
                model_id=self.model_id,
                model_kwargs={
                    "max_tokens": 2000,
                    "temperature": 0.15,
                    "top_p": 0.9,
                }
            )
        except Exception as e:
            print(f"❌ LLM 초기화 실패: {e}")
            return None

In [59]:
llm = AdvancedLLM().llm

In [60]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory

# Your prompt template (already correct)
template_dynamo = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

Relevant Information:

{chat_history}

Conversation:
Human: {question}
AI:"""

prompt_dynamo = PromptTemplate(
    input_variables=["chat_history", "question"], 
    template=template_dynamo
)

# Create chain with DynamoDB history
chain = prompt_dynamo | llm




In [61]:
chain_dynamo = RunnableWithMessageHistory(
    chain,
    get_session_history=lambda session_id: DynamoDBHistory(session_id, "conversations-table"),
    input_messages_key="question",
    history_messages_key="chat_history"
)

# Usage
response = chain_dynamo.invoke(
    {"question": "안녕하세요"},
    config={"configurable": {"session_id": "test-session-001"}}
)



In [62]:
response

AIMessage(content='안녕하세요! 반갑습니다! 😊\n\n한국어로 인사해 주셔서 기쁘네요. 저는 Claude라고 하는 AI 어시스턴트입니다. 오늘 어떻게 지내고 계신가요? \n\n한국어와 영어 모두 편하게 사용하실 수 있으니, 어떤 언어든 편하신 대로 대화하시면 됩니다. 궁금한 것이 있으시거나 도움이 필요한 일이 있으시면 언제든 말씀해 주세요!\n\n혹시 특별히 관심 있는 주제나 이야기하고 싶은 것이 있으신가요?', additional_kwargs={'usage': {'prompt_tokens': 63, 'completion_tokens': 223, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 286}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'global.anthropic.claude-sonnet-4-20250514-v1:0', 'model_name': 'global.anthropic.claude-sonnet-4-20250514-v1:0'}, response_metadata={'usage': {'prompt_tokens': 63, 'completion_tokens': 223, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 286}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'global.anthropic.claude-sonnet-4-20250514-v1:0', 'model_name': 'global.anthropic.claude-sonnet-4-20250514-v1:0'}, id='run--92a6b611-6686-4406-b7fa-171b031adc52-0', usage_metadata={'input_tokens': 63, 'output_tokens'

In [63]:
res1=chain_dynamo.invoke( {"question": "상사병은 무엇인가요?"},
    config={"configurable": {"session_id": "test-session-001"}})

In [64]:
res1

AIMessage(content='안녕하세요! 상사병에 대해 설명해드릴게요! 😊\n\n상사병(相思病)은 한자어로 "서로 그리워하는 병"이라는 뜻으로, 사랑하는 사람을 그리워하며 생기는 마음의 병을 의미합니다. \n\n**상사병의 특징들:**\n\n🔹 **증상들**\n- 사랑하는 사람이 자꾸 생각나서 집중이 안 됨\n- 식욕 부진이나 잠을 제대로 못 잠\n- 가슴이 답답하고 한숨이 자주 나옴\n- 멍하니 있는 시간이 많아짐\n- 그 사람과 관련된 것들에 과도하게 민감해짐\n\n🔹 **문학적 표현**\n- 고전 문학에서 자주 등장하는 소재\n- 춘향전의 이몽룡, 서양의 로미오와 줄리엣 등\n- "그리움"과 "애절함"을 표현하는 시적 장치\n\n🔹 **현대적 관점**\n- 의학적으로는 실제 질병은 아니지만\n- 심리적 스트레스나 우울감으로 이어질 수 있음\n- 짝사랑이나 이별 후에 흔히 경험하는 감정 상태\n\n혹시 상사병과 관련해서 더 궁금한 점이나, 문학 작품 속 사례 등에 대해 더 자세히 알고 싶으신가요?', additional_kwargs={'usage': {'prompt_tokens': 337, 'completion_tokens': 495, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 832}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'global.anthropic.claude-sonnet-4-20250514-v1:0', 'model_name': 'global.anthropic.claude-sonnet-4-20250514-v1:0'}, response_metadata={'usage': {'prompt_tokens': 337, 'completion_tokens': 495, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 

In [65]:
res2=chain_dynamo.invoke( {"question": "나는 상사병이 상사로 인해 스트레스 받아서 생기는 마음의 병이라고 나는생각해"},
    config={"configurable": {"session_id": "test-session-001"}})

In [66]:
res2

AIMessage(content='아하! 정말 재미있는 해석이네요! 😄\n\n말씀하신 것처럼 "상사병"을 직장에서의 **상사(上司) + 병**으로 이해하신 거군요! 사실 이런 식으로 생각하시는 분들이 꽤 많을 것 같아요.\n\n**두 가지 "상사병"을 비교해보면:**\n\n🔹 **제가 설명한 상사병 (相思病)**\n- 한자: 相思病 (서로 상/생각 사/병 병)\n- 의미: 사랑하는 사람을 그리워하는 마음의 병\n\n🔹 **님이 생각하신 상사병 (上司病)**  \n- 한자: 上司病 (위 상/맡을 사/병 병)\n- 의미: 직장 상사 때문에 받는 스트레스성 질환\n\n실제로 직장에서 상사 때문에 스트레스 받아서 생기는 증상들도 있죠:\n- 월요병, 출근 거부감\n- 불면증, 소화불량\n- 우울감, 불안감\n- "상사 얼굴만 봐도 속이 쓰리다" 같은 표현들...\n\n한국어의 동음이의어 특성상 이런 재미있는 오해(?)가 생길 수 있는 것 같아요! 😊\n\n혹시 직장 생활에서 스트레스를 많이 받고 계신 건 아니신가요? 아니면 그냥 단어의 재미있는 해석을 해보신 건가요?', additional_kwargs={'usage': {'prompt_tokens': 934, 'completion_tokens': 503, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 1437}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'global.anthropic.claude-sonnet-4-20250514-v1:0', 'model_name': 'global.anthropic.claude-sonnet-4-20250514-v1:0'}, response_metadata={'usage': {'prompt_tokens': 934, 'completion_tokens': 503, 'cache_read_input_tokens': 0, 'cache_write_in

In [67]:
res3=chain_dynamo.invoke( {"question": "내가 생각하는 상사병은 무엇인가요?"},
    config={"configurable": {"session_id": "test-session-001"}})

In [68]:
res3

AIMessage(content='앞서 대화에서 말씀해주신 내용을 보면, 당신이 생각하는 상사병은:\n\n**상사(上司)로 인해 스트레스 받아서 생기는 마음의 병**이라고 하셨어요! 😊\n\n즉, 직장에서의 **상사 + 병**의 개념으로 이해하고 계시는 거죠. \n\n구체적으로는:\n- 직장 상사 때문에 받는 스트레스\n- 그로 인해 생기는 심리적, 신체적 증상들\n- 상사와의 관계에서 오는 마음의 고통\n\n이렇게 현대적이고 직장인들에게 매우 현실적인 의미로 "상사병"을 해석하고 계신 것 같아요!\n\n전통적인 相思病(그리움의 병)과는 완전히 다른 관점에서, 上司病(상사 때문에 생기는 병)으로 이해하신 거죠. 사실 많은 직장인들이 공감할 수 있는 아주 현실적인 해석이라고 생각해요! 😄\n\n제가 정확히 이해한 게 맞나요?', additional_kwargs={'usage': {'prompt_tokens': 1514, 'completion_tokens': 379, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 1893}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'global.anthropic.claude-sonnet-4-20250514-v1:0', 'model_name': 'global.anthropic.claude-sonnet-4-20250514-v1:0'}, response_metadata={'usage': {'prompt_tokens': 1514, 'completion_tokens': 379, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 1893}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'global.anthropic.claude-sonnet-4-20250